## **GNR 650:** Advanced topics in deep learning for image analysis
### **Assignment 2:** Fine tuning a pre-trained Vision Transformer (B-12) on EuroSAT dataset
> The task is to use a pre-trained ViT like B-12, and prepare four models with the following fine-tuning strategies. Then visualize and compare the intermediate feature responses of all the four models using train/test split of the [EuroSAT](https://www.tensorflow.org/datasets/catalog/eurosat) dataset.
> 1. Fine tuning the final layer 
> 2. Fine tuning the bottom layers 
> 3. Full fine tune on the eurosat data 
> 4. No fine tuning

### Collaborators: 
[![Munish](https://img.shields.io/badge/22M2153-Munish_Monga-blue)](https://github.com/munish30monga)
[![Aniket](https://img.shields.io/badge/22M2162-Aniket_Thomas-darkgreen)](https://github.com/AniTho)

### Table of Contents:
1. [Introduction](#introduction)
2. [Imporing Libraries](#imporing-libraries)
3. [Hyperparameters](#hyperparameters)


### Introduction

### Imporing Libraries

In [1]:
import torch
from torchvision import models
from torchvision import datasets
from torchvision import transforms
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

### Hyperparameters <a class="anchor" id="#hyperparameters"></a>

In [2]:
mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.5, 0.5, 0.5])
inv_mean = -mean
inv_std = 1/std
max_n = 100
batch_size = 32
learning_rate = 0.001
device = 'cuda' if torch.cuda.is_available() else 'cpu'
epochs = 40
seed = 42                   # random seed
print_summary = False